In [1]:
%load_ext autoreload
%autoreload 2

# Getting Started Kaggle TPS Challenge with Tabular ML Toolkit

> A Tutorial to showcase usage of tabular_ml_toolkit (tmlt) library on Kaggle TPS Challenge Nov 2021.

> tabular_ml_toolkit is a helper library to jumpstart your machine learning project based on Tabular or Structured data.

> It comes with model parallelism and cutting edge hyperparameter search techniques.

> Under the hood TMLT uses optuna, xgboost and scikit-learn pipelines

## Install

`pip install -U tabular_ml_toolkit`

### How to Best Use tabular_ml_toolkit

Start with your favorite model and then just simply create **tmlt** with one API.

*Here we are using XGBClassifier, on  [Kaggle TPS Challenge (Nov 2021) data](https://www.kaggle.com/c/tabular-playground-series-nov-2021/data)*

In [ ]:
from tabular_ml_toolkit.tmlt import *
from xgboost import XGBClassifier
import numpy as np
import gc

In [3]:
from sklearn.metrics import roc_auc_score, accuracy_score

In [4]:
# Dataset file names and Paths
DIRECTORY_PATH = "/Users/pamathur/kaggle_datasets/tps_dec_2021/"
TRAIN_FILE = "train.csv"
TEST_FILE = "test.csv"
SAMPLE_SUB_FILE = "sample_submission.csv"
OUTPUT_PATH = "kaggle_tps_dec_output/"

#### Just point tmlt in the direction of your data

#### Let it know what are idx and target columns in your tabular data

#### what kind of problem type you are trying to resolve?

In [5]:
# create tmlt
tmlt = TMLT().prepare_data(
    train_file_path= DIRECTORY_PATH + TRAIN_FILE,
    test_file_path= DIRECTORY_PATH + TEST_FILE,
    #make sure to use right index and target columns
    idx_col="Id",
    target="Cover_Type",
    random_state=42,
    problem_type="multi_class_classification",
#     nrows=4000
)


# tmlt supports only below task type:
    # "binary_classification"
    # "multi_label_classification"
    # "multi_class_classification"
    # "regression"

2021-12-14 15:34:09,623 INFO 12 cores found, model and data parallel processing should worked!
2021-12-14 15:34:51,686 INFO DataFrame Memory usage decreased to 274.66 Mb (83.9% reduction)
2021-12-14 15:35:01,466 INFO DataFrame Memory usage decreased to 67.71 Mb (83.9% reduction)
2021-12-14 15:35:02,218 INFO The least class label is :5 and value count is: 1
2021-12-14 15:35:02,223 INFO The time took to concat 12 rows: 0.0022640228271484375
2021-12-14 15:35:02,223 INFO The X shape BEFORE append is: (4000000, 55)
2021-12-14 15:35:02,502 INFO The time took to append 1 dataframe to existing one!: 0.27852797508239746
2021-12-14 15:35:02,503 INFO The X shape AFTER append is: (4000012, 55)
2021-12-14 15:35:05,110 INFO PreProcessing will include target(s) encoding!
2021-12-14 15:35:05,277 INFO categorical columns are None, Preprocessing will done accordingly!


In [6]:
print(type(tmlt.dfl.X))
print(tmlt.dfl.X.shape)
print(type(tmlt.dfl.y))
print(tmlt.dfl.y.shape)
print(type(tmlt.dfl.X_test))
print(tmlt.dfl.X_test.shape)

<class 'pandas.core.frame.DataFrame'>
(4000012, 54)
<class 'numpy.ndarray'>
(4000012,)
<class 'pandas.core.frame.DataFrame'>
(1000000, 54)


In [7]:
tmlt.dfl.X

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,3189,40,8,30,13,3270,206,234,193,4873,...,0,0,0,0,0,0,0,0,0,0
1,3026,182,5,280,29,3270,233,240,106,5423,...,0,0,0,0,0,0,0,0,0,0
2,3106,13,7,351,37,2914,208,234,137,5269,...,0,0,0,0,0,0,0,0,0,0
3,3022,276,13,192,16,3034,207,238,156,2866,...,0,0,0,0,0,0,0,0,0,0
4,2906,186,13,266,22,2916,231,231,154,2642,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4000007,2953,114,39,97,111,981,181,209,184,7633,...,0,0,0,0,0,0,0,0,0,0
4000008,2953,114,39,97,111,981,181,209,184,7633,...,0,0,0,0,0,0,0,0,0,0
4000009,2953,114,39,97,111,981,181,209,184,7633,...,0,0,0,0,0,0,0,0,0,0
4000010,2953,114,39,97,111,981,181,209,184,7633,...,0,0,0,0,0,0,0,0,0,0


In [8]:
import pandas as pd
print(dict(pd.Series(tmlt.dfl.y).value_counts()))

{1: 2262087, 0: 1468136, 2: 195712, 6: 62261, 5: 11426, 3: 377, 4: 13}


In [ ]:
gc.collect()

### Training


##### create train valid dataframes for quick preprocessing and training

In [9]:
%%time
# create train, valid split to evaulate model on valid dataset
X_train, X_valid,  y_train, y_valid =  tmlt.dfl.create_train_valid(valid_size=0.2)

CPU times: user 1.65 s, sys: 202 ms, total: 1.85 s
Wall time: 1.9 s


In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

# print(X_train.columns.to_list())

(3200009, 54)
(3200009,)
(800003, 54)
(800003,)


In [11]:
# check for class values see if both train and valid have same class labels
print(dict(pd.Series(y_train).value_counts()))
print(dict(pd.Series(y_valid).value_counts()))

{1: 1809818, 0: 1174197, 2: 156714, 6: 49866, 5: 9111, 3: 293, 4: 10}
{1: 452269, 0: 293939, 2: 38998, 6: 12395, 5: 2315, 3: 84, 4: 3}


##### Now PreProcess X_train, X_valid

NOTE: Preprocessing gives back numpy arrays for pandas dataframe

In [12]:
%%time
X_train_np,  X_valid_np = tmlt.pp_fit_transform(X_train, X_valid)

print(type(X_train_np))
print(X_train_np.shape)
# print(X_train_np)
print(type(X_valid_np))
print(X_valid_np.shape)
# print(X_valid_np)
print(type(y_valid))
print(type(y_train))

<class 'numpy.ndarray'>
(3200009, 54)
<class 'numpy.ndarray'>
(800003, 54)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
CPU times: user 5.13 s, sys: 3.19 s, total: 8.31 s
Wall time: 8.69 s


In [ ]:
gc.collect()

#### Create a base xgb classifier model with your best guess params

In [13]:
xgb_params = {
    # your best guess params
    'learning_rate':0.1,
    'eval_metric':'mlogloss',
    # must for xgb classifier otherwise warning will be shown
    'use_label_encoder':False,
    # because 42 is the answer for all the randomness of this universe
    'random_state':42,
    #for GPU
    #'tree_method': 'gpu_hist',
    #'predictor': 'gpu_predictor',
}

xgb_model = XGBClassifier(**xgb_params)

In [16]:
%%time
# Now do model training
xgb_model.fit(X_train_np, y_train,
              verbose=False,
              #detect & avoid overfitting
              eval_set=[(X_valid_np, y_valid)],
              eval_metric="mlogloss",
              early_stopping_rounds=300
             )

KeyboardInterrupt: 

In [ ]:
%%time
#predict
preds = xgb_model.predict(X_valid_np)
preds_probs = xgb_model.predict_proba(X_valid_np)

# Metrics
auc = roc_auc_score(y_valid, preds_probs, multi_class='ovr')
acc = accuracy_score(y_valid, preds)

print(f"AUC is : {auc} while Accuracy is : {acc} ")

In [ ]:
gc.collect()

### For Meta Ensemble Models Training

##### Make sure to PreProcess the data

In [ ]:
%%time
X_np, X_test_np = tmlt.pp_fit_transform(tmlt.dfl.X, tmlt.dfl.X_test)
y_np = tmlt.dfl.y

#### Base Model 1: linear SVM model

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
%%time

# OOF training and prediction on both train and test dataset by a given model
#choose model
linear_oof_model = LinearSVC(tol=1e-7, penalty='l2', dual=False, max_iter=2000, random_state=42)

#fit and predict
linear_oof_model_preds, linear_oof_model_test_preds = tmlt.do_oof_kfold_train_preds(n_splits=5,
                                                                                    model=linear_oof_model,
                                                                                    X = X_np,
                                                                                    y = y_np,
                                                                                    X_test = X_test_np)

if linear_oof_model_preds is not None:
    print(linear_oof_model_preds.shape)

if linear_oof_model_test_preds is not None:    
    print(linear_oof_model_test_preds.shape)

In [ ]:
gc.collect()

#### Base Model 2: Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
%%time

# OOF training and prediction on both train and test dataset by a given model

#choose model
log_oof_model = LogisticRegression(multi_class='multinomial', solver='lbfg', random_state=42)

#fit and predict
log_oof_model_preds, log_oof_model_test_preds = tmlt.do_oof_kfold_train_preds(n_splits=5,
                                                                                    model=log_oof_model,
                                                                                    X = X_np,
                                                                                    y = y_np,
                                                                                    X_test = X_test_np)
if log_oof_model_preds is not None:
    print(log_oof_model_preds.shape)

if log_oof_model_test_preds is not None:    
    print(log_oof_model_test_preds.shape)

In [ ]:
gc.collect()

#### Base Model 3: SKLearn MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
%%time

# OOF training and prediction on both train and test dataset by a given model

#choose model
mlp_oof_model = MLPClassifier(max_iter=1000, early_stopping=True)

#update the model on sklearn pipeline
# tmlt = tmlt.update_model(mlp_oof_model)

# # lets see updated sklearn pipeline with new model
# tmlt.spl

#fit and predict
mlp_oof_model_preds, mlp_oof_model_test_preds = tmlt.do_oof_kfold_train_preds(n_splits=5,
                                                                                    oof_model=mlp_oof_model,
                                                                                    X = X_np,
                                                                                    y = y_np,
                                                                                    X_test = X_test_np)
if mlp_oof_model_preds is not None:
    print(mlp_oof_model_preds.shape)

if mlp_oof_model_test_preds is not None:    
    print(mlp_oof_model_test_preds.shape)

#### Base Model 4: TabNet

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier

In [ ]:
%%time

# OOF training and prediction on both train and test dataset by a given model

#choose model
tabnet_oof_model = TabNetClassifier(optimizer_params=dict(lr=0.02), verbose=0)

#fit and predict
tabnet_oof_model_preds, tabnet_oof_model_test_preds = tmlt.do_oof_kfold_train_preds(n_splits=5,
                                                                                    oof_model=tabnet_oof_model,
                                                                                    X = X_np,
                                                                                    y = y_np,
                                                                                    X_test = X_test_np)

if tabnet_oof_model_preds is not None:
    print(tabnet_oof_model_preds.shape)

if tabnet_oof_model_test_preds is not None:
    print(tabnet_oof_model_test_preds.shape)

#### Now add back based models predictions to X and X_test

In [ ]:
# add based model oof predictions back to X and X_test before Meta model training
tmlt.dfl.X["linear_preds"] = linear_oof_model_preds
tmlt.dfl.X_test["linear_preds"] = linear_oof_model_test_preds

print(tmlt.dfl.X.shape)
print(tmlt.dfl.X_test.shape)

In [ ]:
# add based model oof predictions back to X and X_test before Meta model training
tmlt.dfl.X["log_reg_preds"] = log_oof_model_preds
tmlt.dfl.X_test["log_reg_preds"] = log_oof_model_test_preds

print(tmlt.dfl.X.shape)
print(tmlt.dfl.X_test.shape)

In [ ]:
# add based model oof predictions back to X and X_test before Meta model training
tmlt.dfl.X["mlp_preds"] = mlp_oof_model_preds
tmlt.dfl.X_test["mlp_preds"] = mlp_oof_model_test_preds

print(tmlt.dfl.X.shape)
print(tmlt.dfl.X_test.shape)

In [ ]:
# add based model oof predictions back to X and X_test before Meta model training
tmlt.dfl.X["tabnet_preds"] = tabnet_oof_model_preds
tmlt.dfl.X_test["tabnet_preds"] = tabnet_oof_model_test_preds

print(tmlt.dfl.X.shape)
print(tmlt.dfl.X_test.shape)

In [ ]:
# now just update the tmlt with this new X and X_test

In [ ]:
tmlt = tmlt.update_dfl(X=tmlt.dfl.X, y=tmlt.dfl.y, X_test=tmlt.dfl.X_test)

#### For META Model Training

##### create train valid dataframes for quick preprocessing and training

In [ ]:
%%time
# create train, valid split to evaulate model on valid dataset
X_train, X_valid,  y_train, y_valid =  tmlt.dfl.create_train_valid(valid_size=0.2)

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

# print(X_train.columns.to_list())

##### Now PreProcess X_train, X_valid

NOTE: Preprocessing gives back numpy arrays for pandas dataframe

In [ ]:
%%time
X_train_np,  X_valid_np = tmlt.pp_fit_transform(X_train, X_valid)

print(type(X_train_np))
print(X_train_np.shape)
# print(X_train_np)
print(type(X_valid_np))
print(X_valid_np.shape)
# print(X_valid_np)
print(type(y_valid))
print(type(y_train))

In [ ]:
# xgb_params = {
#     'objective': 'binary:logistic', 
#     'use_label_encoder': False,
#     'n_estimators': 40000,
#     'learning_rate': 0.18515462875481553,
#     'subsample': 0.97, 
#     'colsample_bytree': 0.32,
#     'max_depth': 1,
#     'booster': 'gbtree',
#     'gamma': 0.2, 
#     'tree_method': 'gpu_hist',
#     'reg_lambda': 0.11729916523488974, 
#     'reg_alpha': 0.6318827156945853,
#     'random_state': 42,
#     'n_jobs': 4, 
#     'min_child_weight': 256,
#     #for GPU
# #     'tree_method': 'gpu_hist',
# #     'predictor': 'gpu_predictor',
#     }

In [ ]:
xgb_params = {
    'learning_rate': 0.21761562020600114,
    'eval_metric': 'auc',
    'use_label_encoder': False,
    'random_state': 42,
    'booster': 'gblinear',
    'colsample_bytree': 0.1027132584989078,
    'early_stopping_rounds': 171,
    'max_depth': 6,
    'n_estimators': 7000,
    'reg_alpha': 9.583579660175245e-06,
    'reg_lambda': 9.238315962782784e-05,
    'subsample': 0.4464473710560276,
    'tree_method': 'approx',
    #for GPU
#     'tree_method': 'gpu_hist',
#     'predictor': 'gpu_predictor',
}

In [ ]:
%%time
# Now do model training
xgb_model.fit(X_train_np, y_train,
              verbose=False,
              #detect & avoid overfitting
              eval_set=[(X_train_np, y_train), (X_valid_np, y_valid)],
              eval_metric="auc",
              early_stopping_rounds=300
             )

#predict
preds = xgb_model.predict(X_valid_np)
preds_probs = xgb_model.predict_proba(X_valid_np)[:, 1]

# Metrics
auc = roc_auc_score(y_valid, preds_probs)
acc = accuracy_score(y_valid, preds)

print(f"AUC is : {auc} while Accuracy is : {acc} ")

### WOW!!!!

#### For Meta Model, Let's do Optuna based HyperParameter search to get best params for fit

In [ ]:
# **Just make sure to supply an output directory path so hyperparameter search is saved**
study = tmlt.do_xgb_optuna_optimization(optuna_db_path=OUTPUT_PATH, opt_timeout=60)
print(study.best_trial)

##### now update the meta model with best params from study and then update the sklearn pipeline with this new model

In [ ]:
xgb_params.update(study.best_trial.params)
print("xgb_params", xgb_params)
updated_xgb_model = XGBClassifier(**xgb_params)

#### Let's Use K-Fold Training with best params

In [ ]:
%%time
X_np, X_test_np = tmlt.pp_fit_transform(tmlt.dfl.X, tmlt.dfl.X_test)
y_np = tmlt.dfl.y

In [ ]:
%%time
# k-fold training
xgb_model_metrics_score, xgb_model_test_preds = tmlt.do_kfold_training(X_np,
                                                                       y_np,
                                                                       X_test=X_test_np,
                                                                       n_splits=5,
                                                                       model=updated_xgb_model)

In [ ]:
# predict on test dataset
if xgb_model_test_preds is not None:
    print(xgb_model_test_preds.shape)

In [ ]:
# # take weighted average of both k-fold models predictions
# final_preds = ((0.45 * sci_model_preds) + (0.55* xgb_model_test_preds)) / 2
# print(final_preds.shape)

#### Create Kaggle Predictions

In [ ]:
# sub = pd.read_csv(DIRECTORY_PATH + SAMPLE_SUB_FILE)
# sub['target'] = final_preds
# sub.to_csv('submission.csv', index=False)

In [ ]:
# # hide
# # run the script to build 

# from nbdev.export import notebook2script; notebook2script()